In [1]:
#### Convert coffea output into root TH2's to feed into TUnfold
import coffea
import pickle
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
# import ROOT

In [2]:
%matplotlib inline
fname = "coffeaOutput/trijetHistsTest_QCDsim_pt200.0_eta2.4_bbloose.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )

rootfile = uproot.recreate('trijetHistsQCDsim.root')
result=result[0]
# integrate and sum over axes
rootfile['jet_pt_mass_reco_u'] = result['jet_pt_mass_reco_u'][{'dataset':sum}]
rootfile['jet_pt_mass_reco_g'] = result['jet_pt_mass_reco_g'][{'dataset':sum}]
rootfile['jet_pt_mass_gen_u'] = result['jet_pt_mass_gen_u'][{'dataset':sum}]
rootfile['jet_pt_mass_gen_g'] = result['jet_pt_mass_gen_g'][{'dataset':sum}]
rootfile['mreco_mgen_u'] = result['response_matrix_u'].project("mreco", "mgen")
rootfile['mreco_mgen_g'] = result['response_matrix_g'].project("mreco","mgen")
rootfile['ptreco_ptgen_u'] = result['response_matrix_u'].project("ptreco", "ptgen")
rootfile['ptreco_ptgen_g'] = result['response_matrix_g'].project("ptreco","ptgen")
response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'].project("ptreco", "mreco", "ptgen", "mgen").to_numpy()
response_matrix_g_values = result['response_matrix_g'].project("ptreco", "mreco", "ptgen", "mgen").values()
print(response_matrix_g_values.shape)
ptreco_centers = (ptreco_edges[:-1]+ptreco_edges[1:])/2
print(ptreco_centers.shape)
mreco_centers = (mreco_edges[:-1]+mreco_edges[1:])/2
ptgen_centers = (ptgen_edges[:-1]+ptgen_edges[1:])/2
mgen_centers = (mgen_edges[:-1]+mgen_edges[1:])/2
rootfile['response'] = {'groomed':response_matrix_g_values[np.newaxis],
                       'ungroomed':response_matrix_u_values[np.newaxis]}
rootfile['response_2d'] = {'groomed':response_matrix_g_values.reshape(10*11, 10*21),
                       'ungroomed':response_matrix_u_values.reshape(10*11, 10*21)}
rootfile['centers'] = {'ptreco':ptreco_centers[np.newaxis], 'mreco':mreco_centers[np.newaxis], 
                      'ptgen':ptgen_centers[np.newaxis], 'mgen':mgen_centers[np.newaxis]}
rootfile.close()

print(np.shape(response_matrix_g_values))
print(np.shape(response_matrix_g_values[np.newaxis]))

(10, 11, 10, 21)
(10,)
(10, 11, 10, 21)
(1, 10, 11, 10, 21)


In [3]:
# fname = uproot.open('root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/42B4DEA0-C0E7-A944-80DC-1D107A0F63FB.root')

In [4]:
# print(fname["Events"].keys("*HTXS*"))

In [7]:
hists = uproot.open('trijetHistsQCDsim.root')
print(hists.keys())
print(hists['response'].show())
ungroomed_flat = hists['response_2d']['ungroomed'].array()
ungroomed = hists['response']['ungroomed'].array().to_numpy()
ptreco_centers = hists['centers']['ptreco'].array().to_numpy()
print(np.argwhere(ungroomed==46))
print(np.argwhere(ungroomed.reshape(110,210)==46))
print(np.argwhere(ungroomed.flatten()==46))
print(ungroomed[0][0][4][0][9])


['jet_pt_mass_reco_u;1', 'jet_pt_mass_reco_g;1', 'jet_pt_mass_gen_u;1', 'jet_pt_mass_gen_g;1', 'mreco_mgen_u;1', 'mreco_mgen_g;1', 'ptreco_ptgen_u;1', 'ptreco_ptgen_g;1', 'response;1', 'response_2d;1', 'centers;1']
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
groomed              | double[10][11][10][21]   | AsDtype("('>f8', (10, 11, 1...
ungroomed            | double[10][11][10][21]   | AsDtype("('>f8', (10, 11, 1...
None
[[0 0 4 0 9]]
[[4 9]]
[[849]]
46.0
